In [1]:
cd("C:/Users/jmcke/Desktop/Kaggle NFL")

In [2]:
using CSV, DataFrames, Random, StatsBase

In [3]:
df = CSV.read("train_features_nov7.csv", categorical=true);

In [4]:
df = df[:,2:size(df,2)]
n = size(df,1)
m = size(df,2)
first(df, 5)

,X,Y,S,A,Dis,Orientation,Dir,JerseyNumber
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,46.65,38.83,4.55,0.76,0.51,284.53,169.16,22
2,46.63,18.73,1.24,0.74,0.13,31.48,89.96,23
3,46.09,34.84,1.69,1.13,0.4,278.01,182.82,29
4,50.68,35.42,1.82,2.43,0.16,347.37,195.69,38
5,45.89,16.64,1.11,0.83,0.02,2.77,37.41,39


In [5]:
#binary expansion
for i=1:199
    df[:,m+i] = zeros(n)
    df[df[:,:Yards].==(i-100),m+i] .= 1
end

┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, ::Colon, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[5]:2
└ @ Core .\In[5]:2
┌ Warning: In the future setindex! will disallow adding columnsto a DataFrame using integer index. Use a Symbol to specify a column name instead.
│   caller = setindex!(::DataFrame, ::Array{Float64,1}, ::Colon, ::Int64) at deprecated.jl:67
└ @ DataFrames .\deprecated.jl:67


In [8]:
#remove y label and randomize rows
select!(df, Not(:Yards))
new = df[shuffle(1:size(df, 1)),:];

In [12]:
#generate training and testing sets
l = Int(round(n/50))
train_X = new[1:l,1:m-1]
test_X = new[l+1:n,1:m-1]

train_y = new[1:l,m:size(new,2)]
test_y = new[l+1:n,m:size(new,2)];

In [40]:
#create arrays to fill in later
probs = zeros(n-l)
train_predictions = Float64[]
test_predictions = Float64[];

In [41]:
#create 199 models based on 199 columns (0 1) in the train_y matrix
for i=1:199
    lnr = IAI.OptimalTreeClassifier(random_seed=1,max_depth=5, cp=0.001, minbucket=10)
    IAI.fit!(lnr,train_X,train_y[:,i])
    probs_push = IAI.predict_proba(lnr, test_X) #make predictions on test set
    
    if size(probs_push, 2) > 1
        probs = hcat(probs, IAI.predict_proba(lnr, test_X)[2])
    else
        probs = hcat(probs, zeros(n-l))
    end
end

Training trees...100%|██████████████████████████████████| Time: 0:00:03
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[41]:8
└ @ Core .\In[41]:8
Training trees...100%|██████████████████████████████████| Time: 0:00:05
Training trees...100%|██████████████████████████████████| Time: 0:00:05
Training trees...100%|██████████████████████████████████| Time: 0:00:06
Training trees...100%|██████████████████████████████████| Time: 0:00:07
Training trees...100%|██████████████████████████████████| Time: 0:00:09
Training trees...100%|██████████████████████████████████| Time: 0:00:08
Training trees...100%|██████████████████████████████████| Time: 0:00:09
Training trees...100%|██████████████████████████████████| Time: 0:00:11
Training trees...100%|██████████████████████████████████| Time: 0:00:11
Training trees...100%|██████████████████████████████████| Time: 0:00:11
Training trees...100%|█████████████████████

In [118]:
#create cdf
probs = probs[:,2:size(probs,2)]

cdf = zeros(n,m)
n = size(probs,1)
m = size(probs,2)

for i=1:n
    for j=1:m
        if j==1
            cdf[i,j] = probs[i,j]
        else
            cdf[i,j] = probs[i,j] + cdf[i,(j-1)]
        end
    end
end

#normalize rows 0-1
for i=1:n
    for j=1:m
        cdf[i,j] = cdf[i,j]/cdf[i,m]
    end
end

In [136]:
my_p = Float64[]

#find actual y value for hinge function
for i=1:n
    for j=1:m
        if test_y[i,j]==1
            push!(my_p, j)
        end
    end
end

score = 0
k = 1

#calculate score
for i=1:n
    if Int(my_p[k])!=199 & Int(my_p[k])!=1
        score += (sum((cdf[i,j]-0)^2 for j=1:Int(my_p[k])) + sum((1-cdf[i,j])^2 for j=(Int(my_p[k])+1):m))
    elseif Int(my_p[k])==199
        score += sum((cdf[i,j]-0)^2 for j=1:Int(my_p[k]))
    else
        score += sum((1-cdf[i,j])^2 for j=(Int(my_p[k])+1):m)
    end
    k += 1
end

In [138]:
#final score
println("Our final score is: ", score/(199*n))

Our final score is: 0.013013542638025913


In [135]:
CSV.write("CDF_Kaggle.csv", convert(DataFrame, cdf))

"CDF_Kaggle.csv"